# Calculate robusteness from pi control runs

In [192]:
# Some definitions

# directory of data
#fdir = '~/Google Drive/professional/research/FARALLON_INSTITUTE_PROJECTS/2020 NOAA MAPP/Climate_extremes_sharedfigsandcode/data/FOR SCATTER PLOT/'
fdir = '/Volumes/GoogleDrive/My Drive/Climate_extremes_sharedfigsandcode/data/FOR SCATTER PLOT/'

In [193]:
# Modules
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore') #filter some warning messages
import seaborn as sns

In [194]:
def num_coevents(y1, y2, pct1, pct2, runs='picontrol'):
    # 
    fin1 = fdir+'anomaly ts/'+runs+'_goa_sst_anom.nc'
    fin2 = fdir+'anomaly ts/'+runs+'_sm_anom.nc'
    modsst = xr.open_dataset(fin1)
    modsst.close()
    modsm2 = xr.open_dataset(fin2)
    modsm2.close()
    
    models = modsm2.model

    # selec periood
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))
    
    nev = list()
    
    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values
    
        # calculate threshold
        mhw_thr = np.nanpercentile(tmp1,pct1)
        drg_thr = np.nanpercentile(tmp2,pct2)
    
        a1 = tmp1>=mhw_thr
        a2 = tmp2<=drg_thr
        
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2]
    
        nev.append(len(tmp))#/(y2-y1+1)) #freq.
        
    return nev  # sum(nev)

In [195]:
ny = 30
prc1 = 90
prc2 = 10

pinev=list()
for i in range(int(500/ny)):
    nev = num_coevents(i*ny, (i+1)*ny-1, prc1, prc2)
    pinev.append(nev)
pinev 

[[0, 0, 0, 0, 0, 2, 0, 1, 0, 1, 2, 0, 2, 1, 0, 0, 0, 0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1],
 [0, 2, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 2, 0, 0, 0, 2, 0],
 [0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 2, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 2, 0, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
 [2, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 0,

In [196]:
#model by model

In [197]:
model_mean=np.mean(pinev,axis=0)
model_spread=np.std(pinev,axis=0)
model_mean

array([0.25  , 0.4375, 0.25  , 0.4375, 0.1875, 0.4375, 0.125 , 0.125 ,
       0.3125, 0.125 , 0.4375, 0.5625, 0.375 , 0.4375, 0.1875, 0.5625,
       0.375 , 0.25  , 0.375 , 0.4375, 0.1875, 0.25  ])

In [198]:
#16 30-yr periods MMEs

In [199]:
mme_mean=np.mean(pinev,axis=1)
mme_spread=np.std(pinev,axis=1)
#internal variability mean and spread
mme_mean,np.mean(mme_spread)

(array([0.5       , 0.27272727, 0.36363636, 0.22727273, 0.5       ,
        0.40909091, 0.18181818, 0.36363636, 0.22727273, 0.22727273,
        0.18181818, 0.31818182, 0.40909091, 0.36363636, 0.27272727,
        0.36363636]),
 0.5500570308051732)

In [200]:
#MME mean and spread

In [201]:
mme_model_mean=np.mean(mme_mean)
mme_model_spread=np.std(mme_mean)
mme_model_mean,mme_model_spread

(0.32386363636363635, 0.09890281355414325)

In [202]:
#cross model mean and spread

In [203]:
cross_model_mean=np.mean(model_mean)
cross_model_spread=np.std(model_mean)
cross_model_mean,cross_model_spread

(0.32386363636363635, 0.134095724104613)

In [204]:
## Calculate thresholds for PI Control runs

In [205]:
def get_thre(fdir, runs, y1,y2,prc1,prc2):
    # open data: anomalies (without trend)

    modsst = xr.open_dataset(fdir+'anomaly ts/'+runs+'_goa_sst_anom.nc')
    modsst.close()
    modsm2 = xr.open_dataset(fdir+'anomaly ts/'+runs+'_sm_anom.nc')
    modsm2.close()

    # select period 
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))
    mhw_thr = list()
    drg_thr = list()
    models = list()

    # Calculate thresholds for each model
    for ix,i in enumerate(modsm2.model.values):
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values
        
        mhw_thr.append(np.nanpercentile(tmp1,prc1))
        drg_thr.append(np.nanpercentile(tmp2,prc2))

        models.append(i)
    
    return mhw_thr, drg_thr, models

In [206]:
# calculate number of events for control, historical and future
# based on thresholds in each run

def freq_coevents(fdir, runs, mhw_thr, drg_thr, models, y1, y2):
    #pi control
    fi1 = fdir+'anomaly ts/'+runs+'_goa_sst_anom.nc'
    fi2 = fdir+'anomaly ts/'+runs+'_sm_anom.nc'
    modsst = xr.open_dataset(fi1)
    modsst.close()
    modsm2 = xr.open_dataset(fi2)
    modsm2.close()

    # select period
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))

    nev = list()

    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values

        a1 = tmp1>=mhw_thr[ix]
        a2 = tmp2<=drg_thr[ix]
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2]

        nev.append(len(tmp))
    
    freq = np.array(nev)/(y2-y1+1)
    
    return nev,np.round(np.nanmean(nev),4), np.round(np.nanstd(nev),4) #mean no. of events
#freq, np.round(np.nanmean(freq),4), np.round(np.nanstd(freq),4) #mean frequency


In [207]:
def freq_difruns(fdir,ny, prc1, prc2):
    
    # calculate threshold in pi control
    y2=2014
    y1=y2-ny+1
    mhw_thr, drg_thr, models = get_thre(fdir, 'picontrol',y1,y2,prc1,prc2)

    print ('Percentiles: '+str(prc1)+'/'+str(prc2))
    # pi control
    print('piControl')
    y2=2014
    y1=y2-ny+1
    freq, mfrq, sfrq = freq_coevents(fdir,'picontrol',mhw_thr,drg_thr, models,y1,y2)
    print(mfrq,sfrq)
    
    # calculate threshold in historical 1
    y1 = 1900
    y2 = y1+ny-1
    mhw_thr, drg_thr, models = get_thre(fdir, 'historical',y1,y2,prc1,prc2)

    # historical 1
    y1 = 1900
    y2 = y1+ny-1
    print('\nhistorical: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'historical',mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)

       
    # calculate threshold in historical 2
    y2=2014
    y1=y2-ny+1
    mhw_thr, drg_thr, models = get_thre(fdir, 'historical',y1,y2,prc1,prc2)

    # historical 2
    y2=2014
    y1=y2-ny+1
    print('\nhistorical 2: '+str(y1)+'-'+str(y2))
    freq1, mfrq, sfrq = freq_coevents(fdir,'historical',mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)
    
    # calculate threshold in historical 2
    y2=2099
    y1=y2-ny+1
    mhw_thr, drg_thr, models = get_thre(fdir, 'ssp585',y1,y2,prc1,prc2)

    # future
    y2=2099
    y1=y2-ny+1
    print('\nfuture: '+str(y1)+'-'+str(y2))
    freq2, mfrq, sfrq = freq_coevents(fdir,'ssp585',mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)
    return freq1, freq2

In [208]:
# Frequency of events - detrended data, using individual threshols
## percentiles 90-10

In [209]:

print ('\nPercentiles: '+str(prc1)+'/'+str(prc2))
print('\nProbable # events in '+str(ny)+' years'+': '+str(ny*0.01))

hinev,funev=freq_difruns(fdir,ny, prc1, prc2)
funev


Percentiles: 90/10

Probable # events in 30 years: 0.3
Percentiles: 90/10
piControl
0.0 0.0

historical: 1900-1929
0.5455 0.6556

historical 2: 1985-2014
0.2727 0.5378

future: 2070-2099
0.6364 0.5677


[1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 2, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1]

In [210]:
#16 MMEs diff and spread

In [211]:
diff=np.mean(funev)-mme_mean
diff_mean=np.mean(diff)
diff_spread=np.std(diff)
diff_mean,diff_spread

(0.3125, 0.09890281355414324)

In [212]:
#model by model diff and spread, not right in this case as future only has 1 period, large spread

In [213]:
diff=list()
zip_obj=zip(funev,model_mean)
for i , j in zip_obj:
        diff.append(i-j)
diff

[0.75,
 -0.4375,
 0.75,
 -0.4375,
 -0.1875,
 0.5625,
 0.875,
 -0.125,
 -0.3125,
 0.875,
 0.5625,
 1.4375,
 -0.375,
 0.5625,
 -0.1875,
 0.4375,
 -0.375,
 -0.25,
 0.625,
 0.5625,
 0.8125,
 0.75]

In [214]:
diff_mean=np.mean(diff)
diff_spread=np.std(diff)
diff_mean,diff_spread

(0.3125, 0.5458391163237487)